# Preprocessing Structured Data

Before we actually feed the data into any deep learning system we should look through it carefully. In addition to the kinds of big-picture problems that might arise in collecting data from a noisy world, we need to look out for missing values, strange outliers, and potential errors in the data. The data doesn't have to be completely error free, although obviously that would be best. Frequently, with the size of data we're dealing with, it is not realistic to completely scrub the data of any errors. 

Once we have a collection of data that has a tolerable amount of errors (ideally error free, though that does not HAVE to be the case) we have to transform it into a deep learning friendly format. There are a number of tricks that machine learning practitioners apply to get better results from the same data.

It's also wise to explore the data and look for interesting outliers, correlation between different parts of the data, and other anomolies, oddities, and trends. Of course, we're hoping that our deep learning system can tease these out even better than we could—but that's not a good reason to shirk your own responsibility to understand the dataset. Sophisticated as they are, neural nets are still just tools, and understanding the data can help use hone our tools in the areas where they'll be most successful. 

For this lab we're going to use a public domain dataset from Kaggle. You can find the dataset here:

https://www.kaggle.com/new-york-city/nyc-property-sales

To run this code you'll need to download and unzip that data.

There is useful supporting information about this dataset as well at the following two URLs:

https://www1.nyc.gov/assets/finance/downloads/pdf/07pdf/glossary_rsf071607.pdf

https://www1.nyc.gov/assets/finance/jump/hlpbldgcode.html

The dataset is a record of every building/condo/appartment that was sold in New York City over a 12 month period.

In [1]:
# Pandas is a fantastic and powerful tool for working with structured data
# it's the best of spreadsheets + python, and it has quickly become a go to
# library for data scientists in python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Your path may vary, change this appropriately
path_to_ny_sales = 'datasets/nyc-rolling-sales.csv'

# One of the things we love about pandas is that it's easy to load CSV data
# into a "data frame"
sales_df = pd.read_csv(path_to_ny_sales)

# And, it makes it easy to take a look at the first n items:
sales_df.head(5)

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


In [2]:
# And a summary with bundles of useful information on the numerical fields
sales_df.describe()

,Unnamed: 0,BOROUGH,BLOCK,LOT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,YEAR BUILT,TAX CLASS AT TIME OF SALE
count,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000
mean,10344.359878,2.998758,4237.218976,376.224015,10731.991614,2.025264,0.193559,2.249184,1789.322976,1.657485
std,7151.779436,1.289790,3568.263407,658.136814,1290.879147,16.721037,8.713183,18.972584,537.344993,0.819341
min,4.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4231.000000,2.000000,1322.750000,22.000000,10305.000000,0.000000,0.000000,1.000000,1920.000000,1.000000
50%,8942.000000,3.000000,3311.000000,50.000000,11209.000000,1.000000,0.000000,1.000000,1940.000000,2.000000
75%,15987.250000,4.000000,6281.000000,1001.000000,11357.000000,2.000000,0.000000,2.000000,1965.000000,2.000000
max,26739.000000,5.000000,16322.000000,9106.000000,11694.000000,1844.000000,2261.000000,2261.000000,2017.000000,4.000000


# Removing unwanted columns

We won't always want every piece of data do be considered by our model. Lets delete some columns, and discuss why these might be good candidates for deletion.

In [3]:
sales_df = sales_df.drop(columns=[
    'Unnamed: 0',
    'ADDRESS',          # Hard to parse. Block/zip/borough/neighborhood capture all the value we need. 
    'APARTMENT NUMBER', # Likely irrelevent to the price. Ought to be categorical, which would make data large.
    'SALE DATE',        # Everything was within a 12 month period, likely irrelevant and hard to parse.
    'LOT'               # A lot is a unique identifier within a block, and categorical. Not worth it. 
])

# Look again with dropped columns
sales_df.head(5)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,EASE-MENT,BUILDING CLASS AT PRESENT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,,C2,10009,5,0,5,1633,6440,1900,2,C2,6625000
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,,C7,10009,28,3,31,4616,18690,1900,2,C7,-
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,,C7,10009,16,1,17,2212,7803,1900,2,C7,-
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,,C4,10009,10,0,10,2272,6794,1913,2,C4,3936272
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,,C2,10009,6,0,6,2369,4615,1900,2,C2,8000000


# Fixing erronious datatypes

For a variety of reasons, sometimes the data won't have the datatype we expect when we import it and we'll have to take some corrective measures.

### Micro-exercise: Identify Some Wrongness

Look at the following list of columns and their datatypes. Which of these stand out as potentially erronious? Form a hypthesis about why they might be exhibiting this wrong type.

In [4]:
# Sometimes we get some unexpected datatypes when loading data
for col, dtype in zip(sales_df.columns, sales_df.dtypes):
    print(col, dtype)

BOROUGH int64
NEIGHBORHOOD object
BUILDING CLASS CATEGORY object
TAX CLASS AT PRESENT object
BLOCK int64
EASE-MENT object
BUILDING CLASS AT PRESENT object
ZIP CODE int64
RESIDENTIAL UNITS int64
COMMERCIAL UNITS int64
TOTAL UNITS int64
LAND SQUARE FEET object
GROSS SQUARE FEET object
YEAR BUILT int64
TAX CLASS AT TIME OF SALE int64
BUILDING CLASS AT TIME OF SALE object
SALE PRICE object


In [5]:
# Some of our data that appears numeric really shouldn't be... instead lets treat them as categorical columns.
categorical_columns = [
    'BOROUGH',
    'BLOCK',
    'ZIP CODE',
    'TAX CLASS AT TIME OF SALE'
]

for c in categorical_columns:
    sales_df[c] = sales_df[c].astype('category')

In [6]:
# Lets look at the sale price and see if we can understand why it's not a numeric type...
sales_df['SALE PRICE']

0         6625000
1             -  
2             -  
3         3936272
4         8000000
           ...   
84543      450000
84544      550000
84545      460000
84546    11693337
84547       69300
Name: SALE PRICE, Length: 84548, dtype: object

In [7]:
# Looks like the data is a string type, and sometimes has a value of -
# The documentation suggests the - value means that there was no sale
# just a property transfer for nothing, such as an inheritance. 

# The following operations to obtain "coerced_sales" are only meant
# to evaluate the data. 

# Lets try to coerce the data to numeric where possible:
# coerce "sales_df['SALE PRICE']" and store it into "coerced_sales".
coerced_sales = pd.to_numeric(sales_df['SALE PRICE'], errors='coerce')

# Values that cannot be coerced are changed to Not a Number (NaN). 
# We can use this code to examine those values:
# get only the "sales_df['SALE PRICE']" values where "coerced_sales.isna()"
# is positive. 
only_non_numerics = sales_df['SALE PRICE'][coerced_sales.isna()]

# And this to print all the unique values from only_non_numerics
only_non_numerics.unique()

array([' -  '], dtype=object)

In [8]:
# So, indeed, the only value that wasn't a number as a string was the ' - ' value. 
# good to know. Lets go ahead and coerce them all to zero.

# Coerce the data to numeric where possible repeating the same coercion operation
# as above, but this time on the dataset we need to operate on. 
sales_df['SALE PRICE'] = pd.to_numeric(sales_df['SALE PRICE'], errors='coerce')
sales_df['SALE PRICE'] = sales_df['SALE PRICE'].fillna(0)

In [9]:
sales_df.describe()

,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,YEAR BUILT,SALE PRICE
count,84548.000000,84548.000000,84548.000000,84548.000000,8.454800e+04
mean,2.025264,0.193559,2.249184,1789.322976,1.056623e+06
std,16.721037,8.713183,18.972584,537.344993,1.038794e+07
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,0.000000,0.000000,1.000000,1920.000000,0.000000e+00
50%,1.000000,0.000000,1.000000,1940.000000,4.150000e+05
75%,2.000000,0.000000,2.000000,1965.000000,8.300000e+05
max,1844.000000,2261.000000,2261.000000,2017.000000,2.210000e+09


## Micro-exercise: Dropping missing data instead

The following code demonstrates that the land and gross square feet columns also have missing values. 

Using the documentation here: [https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) write some code that drops all the rows where `LAND SQUARE FEET` has a missing value

In [10]:
# Two other columns should be numeric, but are objects. Lets look at them too to prove they have missing values:
convert_to_numeric = [
    'LAND SQUARE FEET',
    'GROSS SQUARE FEET'
]

for col in convert_to_numeric:
    coerced = pd.to_numeric(sales_df[col], errors='coerce')
    only_non_numerics = sales_df[col][coerced.isna()]

    # And this to print all the unique values from only_non_numerics
    print(col, only_non_numerics.unique())

LAND SQUARE FEET [' -  ']
GROSS SQUARE FEET [' -  ']


In [11]:
# Your code to drop rows here.
sales_df['LAND SQUARE FEET'] = pd.to_numeric(sales_df['LAND SQUARE FEET'], errors='coerce')

# IMPORTANT
# Pay attention at the return type. 
# dataframe.dropna() returns a DataFrame with NA entries 
# dropped from it, so we have to store the result in a variable.
sales_df = sales_df.dropna(subset=['LAND SQUARE FEET'])

sales_df.describe()

,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,YEAR BUILT,SALE PRICE
count,58296.000000,58296.000000,58296.000000,5.829600e+04,58296.000000,5.829600e+04
mean,2.742813,0.279796,3.038167,3.941676e+03,1812.902360,9.817507e+05
std,20.089271,10.492071,22.798842,4.198397e+04,490.972213,1.226303e+07
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,1.000000,0.000000,1.000000,1.650000e+03,1920.000000,0.000000e+00
50%,1.000000,0.000000,1.000000,2.325000e+03,1931.000000,3.780000e+05
75%,2.000000,0.000000,2.000000,3.500000e+03,1960.000000,7.400000e+05
max,1844.000000,2261.000000,2261.000000,4.252327e+06,2017.000000,2.210000e+09


## Solution

In [12]:
# convert first, then drop rows where the value could not be converted to a number
sales_df['LAND SQUARE FEET'] = pd.to_numeric(sales_df[col], errors='coerce')
sales_df = sales_df.dropna(subset=['LAND SQUARE FEET'])

# Note that land square feet now appears in the descriptive stats...
sales_df.describe()

,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,YEAR BUILT,SALE PRICE
count,56930.000000,56930.000000,56930.000000,5.693000e+04,56930.000000,5.693000e+04
mean,2.769015,0.270314,3.055261,4.042317e+03,1845.510434,9.773214e+05
std,20.031627,10.318468,22.672327,3.503270e+04,429.969853,1.234363e+07
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,1.000000,0.000000,1.000000,1.045250e+03,1920.000000,0.000000e+00
50%,1.000000,0.000000,1.000000,1.680000e+03,1931.000000,3.877105e+05
75%,2.000000,0.000000,2.000000,2.560000e+03,1960.000000,7.490000e+05
max,1844.000000,2261.000000,2261.000000,3.750565e+06,2017.000000,2.210000e+09


# Another Alternative for Missing Data: Imputation

Sometimes, we'd rather not just throw out the data in the rest of the rows. If it seems appropriate, we can use a summary statistic like the mean or median in place of the missing values instead of throwing them away. However, use careful judgement when deciding to do this, it may not be appropriate in many contexts.

In this example we'll impute missing values using the mean for the GROSS SQUARE FEET column.

In [ ]:
# So... similarly there are missing values. But, unlike the sale data, we don't have 
# any clues about what this means, and it's hard to imagine that a building exists with
# but occupies zero square feet... We'll apply another common tactic called "imputation"
# We're just going to use the mean value when there is missing data, it's better than nothing
# even though it may be wrong.
from sklearn.impute import SimpleImputer

col = 'GROSS SQUARE FEET'

coerced = pd.to_numeric(sales_df[col], errors='coerce')
sales_df[col] = coerced
sales_df[col] = sales_df[col].astype('float')
    
# Then, we can use the Imputer to fill in any missing values
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean') 

# Only fit it on our relevant column, then transform creates the data for the column with imputed values
# the imputer generally works with 2D data, so we have to comply a bit which is why we have the additional
# square brackets around sales_df[col]
imputer.fit([sales_df[col]]) 
imputed_values = imputer.transform([sales_df[col]])

# Now replace our old Series with the new imputed values. 
# This syntax is more or less coercing a row vector to a column vector.
sales_df['GROSS SQUARE FEET'] = imputed_values[0] # Again, imputed values are 2D...

sales_df.describe()

# Exploring The Data a Bit

Pandas and matplotlib play nice together, and we can easily make some useful charts and figures to help us understand out data.

In [ ]:
# A correlation matrix example:
correlation_matrix = sales_df.corr()
plt.matshow(correlation_matrix)
plt.xticks(range(len(correlation_matrix.columns)), correlation_matrix.columns, rotation='vertical');
plt.yticks(range(len(correlation_matrix.columns)), correlation_matrix.columns);
plt.show()

In [ ]:
# Not surprising that total units seems to correlate most with price. 
# Interesting that residential units seems more correlated than commercial

# What haven't we looked at...
sales_df.columns

In [ ]:
# Hmm, we haven't see anything to do with ease-ment yet. Lets get a sample of what kind of values are in there.
sales_df['EASE-MENT'].unique()

In [ ]:
# It only has one value, junk it, it literally cannot be useful to an ML model if it always has the value ' '. 
sales_df = sales_df.drop(columns=['EASE-MENT'])

In [ ]:
# Lets plot two interesting charts for any numeric column
for col, dtype in zip(sales_df.columns, sales_df.dtypes):
    if dtype not in ['float', 'int', 'float64', 'int64']: continue

    print(col)
    sales_df.boxplot(column=[col])
    sales_df.hist(column=[col])

    plt.tight_layout()
    plt.show()

## Dealing with Outliers?

Something to note about these charts is that, all of our numerical data seems to have a handful of extreme outliers. This might not be a challenge, because they are likely correlated. As in, the building with 1000+ units is probably also one of the sale price outliers. But it does sort of make the histograms unhelpful. 

We could consider pruning these outliers before going ahead with the rest of this data processing. Lets use some rough and tumble outlier detection code from Stack Overflow and replot.

Depending on your goals, it might not make sense to keep the outliers, and because of their status as outliers it is often difficult to train a model to accurately predict this rare events. Often, it makes sense to drop outliers. Doing so in our case definitely makes these plots more useful and interesting too...

In [ ]:
from scipy import stats
import numpy as np

# Lets plot two interesting charts:
for col, dtype in zip(sales_df.columns, sales_df.dtypes):
    if dtype not in ['float', 'int', 'float64', 'int64']: continue

    print(col)
    
    # Quick and dirty outlier filtering, anything over 2 std deviations from the mean 
    # filtered out. 
    filtered_col = sales_df[col][np.abs(stats.zscore(sales_df[col])) < 2]
    
    filtered_col.plot.box()
    plt.show()
    
    filtered_col.hist(bins=10)
    plt.show()

# Cleaning vs Preparing

What we've done above is mostly just cleaning the data. We looked for missing values, and did some spot/sanity checks on our data. We did one thing that you might consider preparing: making some columns categorical. In addition to make sure the data is clean and error free, it's common practice to prepare the data so that it plays nice with neural networks. Two common examples are centering the mean about 0, and normalize the range to be between (0 to 1) or (-1 to 1).

Why? Consider this: year built, square feet, and total units are all going to impact the sale price. One of those might be more impactful than the other, but in the end our neural network is doing a bunch of complex addition and multiplication with those values, but year is always going to be in a range between basically 1900-2017, and units are almost always between 0-50 or so. 1900, when used as a multiplicitive scalar, is going to have a bigger impact than 50. 

For this, and other reasons, it's common to normalize the data so that every datapoint is reduced to it's place within the distribution and to center that distribution between -1 and 1 or 0 and 1. Lets normalize all our numeric values to be between 0 and 1. Note that there are other scaling choices we could make, see the reading resources for this section. 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Note that we are NOT going to scale "sale price" because
# ultimately that will be our target value. WE still need the
# label to be in the format we wish to predict. 
cols_to_scale = [
    'RESIDENTIAL UNITS',
    'COMMERCIAL UNITS',
    'TOTAL UNITS',
    'LAND SQUARE FEET',
    'GROSS SQUARE FEET',
    'YEAR BUILT'
]

scaled_cols = scaler.fit_transform(sales_df[cols_to_scale])

# Wow, was it really that easy? (no)
scaled_cols

In [ ]:
# So, we just got back an NDArray, and we need to put these
# columns back into a dataframe.
scaled_df = sales_df.copy(deep=True)

for i, col  in enumerate(cols_to_scale):
    scaled_df[col] = scaled_cols[:, i]

scaled_df.head(5)

In [ ]:
scaled_df.describe()

# Micro-exercise: Standard Scaling

Min/max scaling and standard scaling are a bit different. Min/max coerces the values into a range of 0-1 and keeps the data's distribution intact. Extracting the original value from the min/max'd value is simple. You store the original min and max values then use the formula `Xi = (Xi - Xmin)/(Xmax-Xmin)`

Another common scaling method is called "Standard Scaling" which coerces the data into a normal distrubtion with its mean centered at 0 with standard deviation of 1.

Using the above code as a guide and this documentation, apply a StandardScalar to this dataframe and then look at the describe results.

In [ ]:
# Your code here

## Solution

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Note that we are NOT going to scale "sale price" because
# ultimately that will be our target value. WE still need the
# label to be in the format we wish to predict. 
cols_to_scale = [
    'RESIDENTIAL UNITS',
    'COMMERCIAL UNITS',
    'TOTAL UNITS',
    'LAND SQUARE FEET',
    'GROSS SQUARE FEET',
    'YEAR BUILT'
]

scaled_cols = scaler.fit_transform(sales_df[cols_to_scale])

# So, we just got back an NDArray, and we need to put these
# columns back into a dataframe.
scaled_df = sales_df.copy(deep=True)

for i, col  in enumerate(cols_to_scale):
    scaled_df[col] = scaled_cols[:, i]

scaled_df.head(5)

# One Hot Encoding with get_dummies

For categorical data, we most likely want to encode it with a one-hot encoding, which pandas makes easy:

In [ ]:
# Even though we labeled some columns as "category" we still need to one-hot 
# encode them. Pandas makes this super easy too:
scaled_dummy_df = pd.get_dummies(scaled_df)
scaled_dummy_df.head(1)

# Note that this takes awhile, it's procecssing a lot of data. 
# Note also that pandas automatically looks for columns with
# a categorical type, so being explicit above was important 
# to making this part easy.

In [ ]:
# Holy crap, 12,413 columns!

# Note that all our numeric columns are between 0 and 1, except SALE PRICE
# All that's left to do here is to separate the labels from the features.

x_train = scaled_dummy_df.drop(columns=['SALE PRICE'])
y_train = scaled_dummy_df['SALE PRICE']

x_train.head(1)

In [ ]:
y_train.head(1)

# For Completeness, Lets Train a Model

This model is dead stupid and fails catastrophically. It is very possible to do better with a more careful curation of the data and a better architecture, but it might or might not be possible to to excellent on this dataset... In the homework exercise you'll be asked to consider this more carefully.

In [ ]:
# Sweet, lets make a simple neural net with keras to make sure we can run the data
# through it. We don't expect great predictions out of this simple model we just
# want to be sure that we can :
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

# Sigmoid and other functions that squash the output might not be
# very appropriate for this task, because our target values are 
# quite large!
model.add(Dense(units=32, activation='relu', input_shape=(len(x_train.columns),)))

# For regression it's common to use a linear activation function
# since our output could be anything. In our case, it would never
# make sense to guess less than 0, so I'm using relu
model.add(Dense(units=1, activation='relu'))

# This function provides useful text data for our network
model.summary()

In [ ]:
# MSE is pretty common for regression tasks
model.compile(optimizer="adam", loss='mean_squared_error')
history = model.fit(x_train, y_train, batch_size=128, epochs=5, verbose=True, validation_split=.2)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='best')

plt.show()

In [ ]:
# So... our model didn't do so great. Okay, it did terribly. 
# It's off by a lot and is clearly overfitting the training data.

# Why might we be getting such poor performance? 

# How could we improve?

# What should we do to the data?

# What about to the model?

In [ ]:
# Two things I would look at:

# Consider discritizing and building a classifier instead of a regressor! 
  # Neural networks tend to be much better at classification tasks.
  # Plus, it's just easier to predict 1 of 10 values compared to a 
  # continious space $0-$100,000,000 or so

# Consider dropping the values with 0 SALE PRICE or any very low value
  # those are not representative of actual sale prices!